In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC



In [3]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
filtered_data3 = data[data['Mtrcyclist'] == 1]
filtered_data3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11227 entries, 3 to 54034
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       11227 non-null  int64  
 1   BedDys           11227 non-null  int64  
 2   BeddystoCC       11227 non-null  float64
 3   Severity         11227 non-null  int64  
 4   DiedCases        11227 non-null  int64  
 5   DiedBdDys        11227 non-null  int64  
 6   Hsptlsns         11227 non-null  int64  
 7   April            11227 non-null  int64  
 8   August           11227 non-null  int64  
 9   December         11227 non-null  int64  
 10  February         11227 non-null  int64  
 11  January          11227 non-null  int64  
 12  July             11227 non-null  int64  
 13  June             11227 non-null  int64  
 14  March            11227 non-null  int64  
 15  May              11227 non-null  int64  
 16  November         11227 non-null  int64  
 17  October          

In [5]:
num_bins = 2

bed_days_min = filtered_data3['Severity'].min()
bed_days_max = filtered_data3['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data3['Bed_days_category'] = filtered_data3['Severity'].apply(bin_function)


/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_1880/513295569.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data3['Bed_days_category'] = filtered_data3['Severity'].apply(bin_function)


In [6]:
from xgboost import XGBClassifier

X = filtered_data3[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data3['Bed_days_category']

XGBOOST 

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before calibration
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(xgb_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predictions and probabilities after calibration
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    confusion_matrices_after.append(confusion_matrix(y_test, y_pred_after))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Calibration:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_after)} After Calibration:\n', confusion_matrices_after[-1])

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix before and after calibration
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0).astype(int)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0).astype(int)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Confusion Matrix for Fold 1 Before Calibration:
 [[1080   10]
 [   7   26]]
Confusion Matrix for Fold 1 After Calibration:
 [[1064   26]
 [   1   32]]
Confusion Matrix for Fold 2 Before Calibration:
 [[1083    7]
 [  10   23]]
Confusion Matrix for Fold 2 After Calibration:
 [[1059   31]
 [   1   32]]
Confusion Matrix for Fold 3 Before Calibration:
 [[1077   13]
 [   2   31]]
Confusion Matrix for Fold 3 After Calibration:
 [[1057   33]
 [   1   32]]
Confusion Matrix for Fold 4 Before Calibration:
 [[1078   12]
 [   6   27]]
Confusion Matrix for Fold 4 After Calibration:
 [[1066   24]
 [   2   31]]
Confusion Matrix for Fold 5 Before Calibration:
 [[1080   10]
 [   9   24]]
Confusion Matrix for Fold 5 After Calibration:
 [[1067   23]
 [   2   31]]
Confusion Matrix for Fold 6 Before Calibration:
 [[1081    9]
 [  11   22]]
Confusion Matrix for Fold 6 After Calibration:
 [[1061   29]
 [   1   32]]
Confusion Matrix for Fold 7 Before Calibration:
 [[1086    4]
 [   9   24]]
Confusion Matrix f

LOGESTIC REGRESSION

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    lr_model = LogisticRegression( random_state=42)
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_lr_before = lr_model.predict(X_test)
    y_prob_lr_before = lr_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_lr_before))
    precision_list_before.append(precision_score(y_test, y_pred_lr_before))
    recall_list_before.append(recall_score(y_test, y_pred_lr_before))
    f1_list_before.append(f1_score(y_test, y_pred_lr_before))
    entropy_list_before.append(calculate_entropy(y_prob_lr_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_lr_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(lr_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_lr_after = calibrated_model.predict(X_test)
    y_prob_lr_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_lr_after))
    precision_list_after.append(precision_score(y_test, y_pred_lr_after))
    recall_list_after.append(recall_score(y_test, y_pred_lr_after))
    f1_list_after.append(f1_score(y_test, y_pred_lr_after))
    entropy_list_after.append(calculate_entropy(y_prob_lr_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_lr_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.983878965655719
Accuracy Std Dev: 0.004368340521579372
Mean Precision: 0.760212024246507
Precision Std Dev: 0.09447873632901985
Mean Recall: 0.6636363636363638
Recall Std Dev: 0.08068804215572332
Mean F1-score: 0.7073873423280795
F1-score Std Dev: 0.08089580180919448
Mean Entropy: 0.0461581779053999
Entropy Std Dev: 0.002692786060779342
Mean Confusion Matrix:
 [[1082.7    7. ]
 [  11.1   21.9]]
--- After Calibration ---
Mean Accuracy: 0.97817859597494
Accuracy Std Dev: 0.004781101798272854
Mean Precision: 0.5875988917412576
Precision Std Dev: 0.07074232691496928
Mean Recall: 0.9272727272727274
Recall Std Dev: 0.0388068135601991
Mean F1-score: 0.7168713889712778
F1-score Std Dev: 0.05146810683613162
Mean Entropy: 0.0319046372992298
Entropy Std Dev: 0.0022568746214048307
Mean Confusion Matrix:
 [[1067.6   22.1]
 [   2.4   30.6]]


Decesion tree

In [9]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_dt_before = dt_model.predict(X_test)
    y_prob_dt_before = dt_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_dt_before))
    precision_list_before.append(precision_score(y_test, y_pred_dt_before))
    recall_list_before.append(recall_score(y_test, y_pred_dt_before))
    f1_list_before.append(f1_score(y_test, y_pred_dt_before))
    entropy_list_before.append(calculate_entropy(y_prob_dt_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_dt_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(dt_model, method='sigmoid', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_dt_after = calibrated_model.predict(X_test)
    y_prob_dt_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_dt_after))
    precision_list_after.append(precision_score(y_test, y_pred_dt_after))
    recall_list_after.append(recall_score(y_test, y_pred_dt_after))
    f1_list_after.append(f1_score(y_test, y_pred_dt_after))
    entropy_list_after.append(calculate_entropy(y_prob_dt_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_dt_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9788015295165261
Accuracy Std Dev: 0.0032044571758565246
Mean Precision: 0.6742287144586917
Precision Std Dev: 0.08227501839086661
Mean Recall: 0.5575757575757575
Recall Std Dev: 0.09791208740244552
Mean F1-score: 0.604612594250433
F1-score Std Dev: 0.06760944002055104
Mean Entropy: 2.402585101263083e-09
Entropy Std Dev: 3.460344766235445e-25
Mean Confusion Matrix:
 [[1080.5    9.2]
 [  14.6   18.4]]
--- After Calibration ---
Mean Accuracy: 0.9800481902467132
Accuracy Std Dev: 0.0029862599818735163
Mean Precision: 0.7146767363071711
Precision Std Dev: 0.07900526861391224
Mean Recall: 0.5454545454545456
Recall Std Dev: 0.08243315459839665
Mean F1-score: 0.614578925368107
F1-score Std Dev: 0.06414035608054117
Mean Entropy: 0.08459527927396888
Entropy Std Dev: 0.002352100233583976
Mean Confusion Matrix:
 [[1082.3    7.4]
 [  15.    18. ]]


SVM

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base SVM model
    svm_model = SVC(probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = svm_model.predict(X_test)
    y_prob_before = svm_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(svm_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9821859578446454
Accuracy Std Dev: 0.0026708961756226196
Mean Precision: 0.747856363837401
Precision Std Dev: 0.10053306345820061
Mean Recall: 0.6303030303030303
Recall Std Dev: 0.11642044068059726
Mean F1-score: 0.6715764147405939
F1-score Std Dev: 0.055696138707990415
Mean Entropy: 0.04459683062940397
Entropy Std Dev: 0.0018270809916379495
Mean Confusion Matrix:
 [[1081.9    7.8]
 [  12.2   20.8]]

After Calibration:
Mean Accuracy: 0.983077382171196
Accuracy Std Dev: 0.003941216444291498
Mean Precision: 0.6827695926533137
Precision Std Dev: 0.09071318515292762
Mean Recall: 0.8272727272727274
Recall Std Dev: 0.057575757575757586
Mean F1-score: 0.7440715196655229
F1-score Std Dev: 0.052711811876991706
Mean Entropy: 0.04604654957439826
Entropy Std Dev: 0.0035833168970581825
Mean Confusion Matrix:
 [[1076.4   13.3]
 [   5.7   27.3]]


Random Forest

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(rf_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9820080221840213
Accuracy Std Dev: 0.0019810051095721874
Mean Precision: 0.7874326098325768
Precision Std Dev: 0.0790632766224735
Mean Recall: 0.5454545454545455
Recall Std Dev: 0.07545393695750746
Mean F1-score: 0.6384270323797488
F1-score Std Dev: 0.04918095495164214
Mean Entropy: 0.04125179538100012
Entropy Std Dev: 0.003698420032414755
Mean Confusion Matrix:
 [[1084.5    5.2]
 [  15.    18. ]]

After Calibration:
Mean Accuracy: 0.9792470035856973
Accuracy Std Dev: 0.003032503670495025
Mean Precision: 0.6160209677133476
Precision Std Dev: 0.06581807429275474
Mean Recall: 0.8212121212121213
Recall Std Dev: 0.06562547826578095
Mean F1-score: 0.7001781630147746
F1-score Std Dev: 0.0359303951856061
Mean Entropy: 0.04331270753368525
Entropy Std Dev: 0.003104323008461269
Mean Confusion Matrix:
 [[1072.3   17.4]
 [   5.9   27.1]]
